# Проект 2 Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод

Дана объёмная база данных, наша целевая задача разобраться в том влияет ли семейное положение, количество детей клиента, цель ради которой берёться кредит, а так же уровень дохода на факт погашения кредита в срок.
Сразу видно, благодаря вызову info() что в стобце total_income есть пропущенные/нулевые значения, нам обязательно нужно предобработать данные по этому столбцу, так как качество предопработки повлияет на точность ответа на целевой вопрос.



### Шаг 2. Предобработка данных

Обработка пропусков

In [146]:
# В данных мусор, сперва нужно понять что можно подставить в пропуски
                # По столбцу days_employed
# Уберём отрицательные значения, сперва уберём знак "минус":
data.loc[data['days_employed'] < 0, 'days_employed'] = data.loc[data['days_employed'] < 0, 'days_employed']*(-1)
# Трудовой стаж в среднем случае не может быть больше количетсва прожитых лет после 18, попробуем сделать фильтрованную таблицу:
#data['years_of_working_age']= data['dob_years']-18
#data['years_employed'] = data['days_employed']/365
#Предположим, что вводились отработанные часы, попробуем аномальные значения разделить на 8 - так как это средний рабочий день
#data.loc[data['years_of_working_age']>data['years_employed'],'days_employed'] = data['days_employed']/8
#print(data)
# по прежнему много аномальных значений

# попробуем сделать фильтрованную таблицу, где будут только нормальные данные о стаже:
#mean_filtred_data=filtred_data.groupby('dob_years')['days_employed'].mean()/365
#print(mean_filtred_data.groupby('dob_years')['years_employed'].mean())
#Создана таблица с неаномальными данными, там по прежнему плохие средние значения, теперь слишком маленький трудовой стаж
#Недостаточно данных, чтобы заменить плохие данные на адекватные
#радует что столбец не нужен для формирования вывода для поставленной цели
# Единственное предположение, это то что человек не работал.
data['days_employed'] = data['days_employed'].fillna(0)


    # По столбцу total_income
#Доходы зависят больше от сферы работы
#print(data.groupby('income_type')['total_income'].mean())
#У безработного обычно не бывает дохода:
data.loc[data['income_type']=='безработный','total_income'] = 0
#print(data[data['income_type'].isna()])

#data['total_income'] = data['total_income'].fillna(1)
#print(data[data['total_income']==1])
#filtred2_data=data[data['total_income']==1]
#print(filtred_data['income_type'].value_counts())
#в глаза бросается, то что, все у кого NaN в доходе - NaN в количестве отработанных дней
#Единого признака, клиентов банка, с неуказанными доходам - не обнаружено, однако все профессии - дожны иметь доход

#Главная цель - сохранить среднестатистическое значение total_income для всей таблицы, для формирования вывода основанного на среднестатических данных
#print(data['total_income'].mean(),data['total_income'].median())
# медиана меньше - заменим на нёё
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

Вывод
info говорит о том что пропуски есть в столбцах 'days_employed' и 'total_income'
Пропуски в обоих столбцах в равных количествах, скорее всего они образовались из-за того, что не заполняли эту графу.

По столбцу days_employed:
Недостаточно данных, чтобы заменить плохие данные на адекватные, так как для целей задачи столцеб days_employed - не нужен:
Нет ни какого смысла тратить время, столбец не нужен для ответа на поставленный вопрос в задаче
Для банка - такой мусор в данных огромная проблема для создания хорошей скоринговой модели
Причины появления такого мусара не ясны - но в банке они точно должны исправить причину, чтобы получать адекватные данные

По столбцу total_income:
В нужном для выполнения поставленной задачи столбце total_income заменены пропуски на медиану


В нужном для выполнения поставленной задачи столбце total_income заменены пропуски на медиану
Для столбца days_employed не найденно адекватного метода, чтобы поставить адекватные данные

Так как нет единого признака - почему образовались пропуски и много мусора в данных, можно сделать вывод:
Что банк использует плохую форму для ввода данных, её нужно было стандартизировать, не позволять вносить аномальные данные:
такие как отрицательное  количество отработанных дней, аномально огромное количество отработанных дней и тд

### Замена типа данных

In [147]:
#Есть ли зависимость между наличием детей и возвратом кредита в срок?
#Есть ли зависимость между семейным положением и возвратом кредита в срок?
#Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
#Как разные цели кредита влияют на его возврат в срок?

# по заданию нужно заменить вещественный тип данных на целочисленный

#children            21525 non-null int64
#days_employed       19351 non-null float64 - нужно заменить
#dob_years           21525 non-null int64
#education           21525 non-null object - не будем трогать, так как является текстом
#education_id        21525 non-null int64
#family_status       21525 non-null object - не будем трогать, так как является текстом
#family_status_id    21525 non-null int64
#gender              21525 non-null object - не будем трогать, так как является текстом
#income_type         21525 non-null object - не будем трогать, так как является текстом
#debt                21525 non-null int64
#total_income        19351 non-null float64 - нужно заменить
#purpose             21525 non-null object - не будем трогать, так как является текстом


data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')

### Вывод

Мы заменили тип данных в столбцах, где это было можно на челочисленный
Не во всех столбцах вещественный тип данных изменён на целочисленный, так как там текст

### Обработка дубликатов

In [148]:
#Приведём данные в более единый формат, методом приведения к нижнему регистру
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()

data = data.drop_duplicates().reset_index(drop = True) 
# аргумент drop со значением True, # чтобы не создавать столбец со # старыми значениями индексов
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

В данных имелись строки написанные с зажатым капслоком, нужно было привести данные к нижнему регистру
И после этого проводить дедубликацию - мы так и сделали
После этого количество строк - уменьшилось

### Лемматизация

In [149]:
from pymystem3 import Mystem
m = Mystem()

def add_lemmatize_purpose(data):
    lemmas = m.lemmatize(data['purpose'])
    return lemmas
data['lemmatize_purpose'] = data.apply(add_lemmatize_purpose, axis=1)
print('сумма всех найденных уникальных значений',data['lemmatize_purpose'].value_counts().sum())
print(data['lemmatize_purpose'].value_counts())

#основные цели кредита:
# 'автомобиль' , 'свадьба', 'жильё' , 'недвижимость' , 'образование' 
# жильё и недвижимость - один тип

сумма всех найденных уникальных значений 21454
[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жило

### Вывод
#основные цели кредита:

'автомобиль' , 'свадьба', 'жильё' , 'недвижимость' , 'образование' 

жильё и недвижимость - один тип

### Категоризация данных

In [183]:
#основные цели кредита:
# 'автомобиль' , 'свадьба', 'жильё' , 'недвижимость' , 'образование' 
# жильё и недвижимость - один тип

def add_type_purpose(data):
    lemmatize_purpose = data['lemmatize_purpose']
    if ('автомобиль') in lemmatize_purpose:
        return 'автомобиль'
    if ('свадьба') in lemmatize_purpose:
        return 'свадьба'
    if ('жилье') in lemmatize_purpose or ('недвижимость') in lemmatize_purpose:
        return 'недвижимость'
    if 'образование' in lemmatize_purpose:
        return 'образование'
    else: return lemmatize_purpose
#
data['type_purpose'] = data.apply(add_type_purpose, axis=1)
print('сумма всех найденных уникальных значений',data['type_purpose'].value_counts().sum())
print(data['type_purpose'].value_counts())

#Есть ли зависимость между наличием детей и возвратом кредита в срок? №
#Есть ли зависимость между семейным положением и возвратом кредита в срок? №
#Есть ли зависимость между уровнем дохода и возвратом кредита в срок? №
#Как разные цели кредита влияют на его возврат в срок? №

min_income = data['total_income'].min() #20 667
median_income = data['total_income'].median() #145 017
mean_income = data['total_income'].mean() #165 209
max_income = data['total_income'].max() #226 5604

#Нужно типизировать тип дохода на низкий, средний, высокий
def add_type_total_income(data):
    income = data['total_income']
    if income < median_income:
        return 'Низкий'
    if median_income < income > mean_income:
        return 'Средний'
    if income < mean_income:
        return 'Высокий'
data['type_total_income'] = data.apply(add_type_total_income, axis=1)

#Нужно типизировать наличие детей
def add_type_have_children(data):
    children = data['children']
    if children == 0:
        return 'Не имеет детей'
    else:
        return 'Имеет детей'
data['type_have_children'] = data.apply(add_type_have_children, axis=1)

#print(data)        

сумма всех найденных уникальных значений 21454
недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: type_purpose, dtype: int64


### Вывод

В итоге, после категоризации данных - у нас получилось выделить 4 основные цели взятия кредита: 
Цель - "недвижимость" - у 10814 клиентов
Цель - "автомобиль" у 4308 клиентов
Цель - "образование" у 4014 клиентов
Цель - "свадьба" у 2335 клиентов

Так же выделены типы дохода: низкий, средний, высокий

И выделено наличие признака имеют или не имеют детей

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [186]:
#print(data['children'].value_counts()) 

klient = data.count()
klient_sum = klient[1]
debt_klient = data[data['debt'] == 1].count()
debt_klient_sum = debt_klient[1]
coefficient_debt = debt_klient_sum / klient_sum
print('Средний процент невозрата составляет {:.2%}'.format(coefficient_debt))

debt_dont_have_children = data[(data['children'] == 0) & (data['debt'] == 1)].count()
dont_have_children = data[data['children'] == 0].count()
sum_debt_dont_have_children = debt_dont_have_children[1]
sum_dont_have_children = dont_have_children[1]
coefficient_debt_dont_have_children = sum_debt_dont_have_children/sum_dont_have_children
print('Процент невозрата клиентов без детей составляет {:.2%}'.format(coefficient_debt_dont_have_children))


debt_have_children = data[(data['children'] != 0) & (data['debt'] == 1)].count()
have_children = data[data['children'] != 0].count()
sum_debt_have_children = debt_have_children[1]
sum_have_children = have_children[1]
coefficient_debt_have_children = sum_debt_have_children/sum_have_children
print('Процент невозрата клиентов с детьми составляет {:.2%}'.format(coefficient_debt_have_children))

data_select_children = [
    ['Должники','Без детей {:.2%}'.format(coefficient_debt_dont_have_children),'Лучше на {:.2%}'.format((coefficient_debt_dont_have_children-coefficient_debt)*-1)],
    ['Должники','С детьми {:.2%}'.format(coefficient_debt_have_children), 'Хуже на {:.2%}'.format(coefficient_debt_have_children-coefficient_debt)]

]
select_children_columns = ['Категория','Подкатегория','Отклонение от среднего']
df_data_select_children = pd.DataFrame(data = data_select_children, columns = select_children_columns)

print(df_data_select_children)

#data_pivot_children = data_final.pivot_table(index=['type_have_children'], columns='debt', values='debt', aggfunc='sum')
#print(data_pivot_children)

Средний процент невозрата составляет 8.12%
Процент невозрата клиентов без детей составляет 7.54%
Процент невозрата клиентов с детьми составляет 9.21%
  Категория     Подкатегория Отклонение от среднего
0  Должники  Без детей 7.54%         Лучше на 0.57%
1  Должники   С детьми 9.21%          Хуже на 1.09%


### Вывод

Средний процент невозрата составляет 8.11%
Процент невозрата клиентов с детьми составляет 9.21% - клиенты с детьми имеют процент невозрата хуже среднего

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [152]:
#print(data['family_status_id'].value_counts())
#print(data['family_status'].value_counts())

# 0 - женат / замужем
# 1 - гражданский брак
# 4 - Не женат / не замужем
# 3 - в разводе
# 2 - вдовец / вдова

print('Средний процент невозрата составляет {:.2%}'.format(coefficient_debt))

id_family_0 = data[data['family_status_id']==0].count()
sum_id_family_0 = id_family_0[1]
debt_id_family_0 = data[ (data['family_status_id']==0) & (data['debt'] == 1) ].count()
sum_debt_id_family_0 = debt_id_family_0[1]
coefficient_debt_id_family_0 = sum_debt_id_family_0 / sum_id_family_0

id_family_1 = data[data['family_status_id']==1].count()
sum_id_family_1 = id_family_1[0]
debt_id_family_1 = data[ (data['family_status_id']==1) & (data['debt'] == 1) ].count()
sum_debt_id_family_1 = debt_id_family_1[1]
coefficient_debt_id_family_1 = sum_debt_id_family_1 / sum_id_family_1

id_family_2 = data[data['family_status_id']==2].count()
sum_id_family_2 = id_family_2[0]
debt_id_family_2 = data[ (data['family_status_id']==2) & (data['debt'] == 1) ].count()
sum_debt_id_family_2 = debt_id_family_2[1]
coefficient_debt_id_family_2 = sum_debt_id_family_2 / sum_id_family_2

id_family_3 = data[data['family_status_id']==3].count()
sum_id_family_3 = id_family_3[0]
debt_id_family_3 = data[ (data['family_status_id']==3) & (data['debt'] == 1) ].count()
sum_debt_id_family_3 = debt_id_family_3[1]
coefficient_debt_id_family_3 = sum_debt_id_family_3 / sum_id_family_3

id_family_4 = data[data['family_status_id']==4].count()
sum_id_family_4 = id_family_4[0]
debt_id_family_4 = data[ (data['family_status_id']==4) & (data['debt'] == 1) ].count()
sum_debt_id_family_4 = debt_id_family_4[1]
coefficient_debt_id_family_4 = sum_debt_id_family_4 / sum_id_family_4



print('Процент невозрата клиентов женат / замужем составляет {:.2%}'.format(coefficient_debt_id_family_0))
print('Процент невозрата клиентов гражданский брак составляет {:.2%}'.format(coefficient_debt_id_family_1))
print('Процент невозрата клиентов вдовец / вдова составляет {:.2%}'.format(coefficient_debt_id_family_2))
print('Процент невозрата клиентов в разводе составляет {:.2%}'.format(coefficient_debt_id_family_3))
print('Процент невозрата клиентов Не женат / не замужем составляет {:.2%}'.format(coefficient_debt_id_family_4))

# 0 - женат / замужем
# 1 - гражданский брак
# 2 - вдовец / вдова
# 3 - в разводе
# 4 - Не женат / не замужем

data_sellect_id_family = [
    ['С семейным положением','в браке {:.2%}'.format(coefficient_debt_id_family_0), 'Лучше на {:.2%}'.format((coefficient_debt_id_family_0-coefficient_debt)*-1)],
    ['С семейным положением','В гражданском браке {:.2%}'.format(coefficient_debt_id_family_1), 'Хуже на {:.2%}'.format(coefficient_debt_id_family_1-coefficient_debt)],
    ['С семейным положением','Вдовец/вдова {:.2%}'.format(coefficient_debt_id_family_2), 'Лучше на {:.2%}'.format((coefficient_debt_id_family_2-coefficient_debt)*-1)],
    ['С семейным положением','В разводе {:.2%}'.format(coefficient_debt_id_family_3), 'Лучше на {:.2%}'.format((coefficient_debt_id_family_3-coefficient_debt)*-1)],
    ['С семейным положением','Вне брака {:.2%}'.format(coefficient_debt_id_family_4), 'Хуже на {:.2%}'.format((coefficient_debt_id_family_4-coefficient_debt)*1)]
    
]

sellect_id_family_children_columns = ['Категория','Подкатегория','Отклонение от среднего']
df_data_sellect_id_family= pd.DataFrame(data = data_sellect_id_family, columns = sellect_id_family_children_columns)
print(df_data_sellect_id_family)

Средний процент невозрата составляет 8.12%
Процент невозрата клиентов женат / замужем составляет 7.55%
Процент невозрата клиентов гражданский брак составляет 9.35%
Процент невозрата клиентов вдовец / вдова составляет 6.57%
Процент невозрата клиентов в разводе составляет 7.11%
Процент невозрата клиентов Не женат / не замужем составляет 9.75%
               Категория               Подкатегория Отклонение от среднего
0  С семейным положением              в браке 7.55%         Лучше на 0.57%
1  С семейным положением  В гражданском браке 9.35%          Хуже на 1.23%
2  С семейным положением         Вдовец/вдова 6.57%         Лучше на 1.55%
3  С семейным положением            В разводе 7.11%         Лучше на 1.00%
4  С семейным положением            Вне брака 9.75%          Хуже на 1.64%


Вывод:
Средний процент невозрата составляет 8.11% 
Процент невозрата клиентов женат / замужем составляет 7.54% - лучше среднего
Процент невозрата клиентов гражданский брак составляет 9.32% - хуже среднего
Процент невозрата клиентов вдовец / вдова составляет 6.57% - лучше среднего
Процент невозрата клиентов в разводе составляет 7.11% - лучше среднего 
Процент невозрата клиентов Не женат / не замужем составляет 9.75% - лучше среднего

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [153]:
#print(data['total_income'].value_counts())
min_income = data['total_income'].min() #20 667
median_income = data['total_income'].median() #145 017
mean_income = data['total_income'].mean() #165 209
max_income = data['total_income'].max() #226 5604
print('Средний процент невозрата составляет {:.2%}'.format(coefficient_debt))

income_low = data[data['total_income'] < median_income].count()
sum_income_low = income_low[1]
debt_income_low = data[ (data['total_income'] < median_income) & (data['debt'] == 1) ].count()
sum_debt_income_low =debt_income_low[1]
coefficient_debt_income_low = sum_debt_income_low / sum_income_low
print('Процент невозрата клиентов с доходом ниже среднего составляет {:.2%}'.format(coefficient_debt_income_low))

income_median = data[ (data['total_income'] > median_income) & (data['total_income'] < mean_income)].count()
sum_income_median = income_median[1]
debt_income_median = data[ (data['total_income'] > median_income) & (data['total_income'] < mean_income) & (data['debt'] == 1) ].count()
sum_debt_income_median = debt_income_median[1]
coefficient_debt_income_median = sum_debt_income_median / sum_income_median
print('Процент невозрата клиентов c доходом выше среднего составляет {:.2%}'.format(coefficient_debt_income_median))

income_high = data[data['total_income'] > mean_income].count()
sum_income_high = income_high[1]
debt_income_high = data[ (data['total_income'] > mean_income) & (data['debt'] == 1) ].count()
sum_debt_income_high =debt_income_high[1]
coefficient_debt_income_high = sum_debt_income_high / sum_income_high
print('Процент невозрата клиентов с высоким доходом составляет {:.2%}'.format(coefficient_debt_income_high))

data_sellect_income = [
    ['C доходом','ниже среднего {:.2%}'.format(coefficient_debt_income_low), 'Хуже на {:.2%}'.format((coefficient_debt_income_low-coefficient_debt)*1)],
    ['C доходом','среднего уровня {:.2%}'.format(coefficient_debt_income_median), 'Хуже на {:.2%}'.format((coefficient_debt_income_median-coefficient_debt)*1)],
    ['C доходом','выше среднего {:.2%}'.format(coefficient_debt_income_high), 'Лучше на {:.2%}'.format((coefficient_debt_income_high-coefficient_debt)*-1)]
]
sellect_income_columns = ['Категория','Подкатегория','Отклонение от среднего']
df_data_sellect_income= pd.DataFrame(data = data_sellect_income, columns = sellect_income_columns)
print(df_data_sellect_income)

Средний процент невозрата составляет 8.12%
Процент невозрата клиентов с доходом ниже среднего составляет 8.31%
Процент невозрата клиентов c доходом выше среднего составляет 8.67%
Процент невозрата клиентов с высоким доходом составляет 7.73%
   Категория           Подкатегория Отклонение от среднего
0  C доходом    ниже среднего 8.31%          Хуже на 0.20%
1  C доходом  среднего уровня 8.67%          Хуже на 0.56%
2  C доходом    выше среднего 7.73%         Лучше на 0.38%


### Вывод

Средний процент невозрата составляет 8.11%
Процент невозрата клиентов с доходом ниже среднего составляет 8.31% - хуже среднего
Процент невозрата клиентов выше среднего составляет 8.63% - хуже среднего 
Процент невозрата клиентов с доходом выше среднего составляет 7.74% - лучше среднего

- Как разные цели кредита влияют на его возврат в срок?

In [182]:
type_reit = data[data['type_purpose'] == 'недвижимость'].count()
sum_type_reit = type_reit[1]
debt_type_reit = data[(data['type_purpose'] == 'недвижимость')&(data['debt'] == 1)].count()
sum_debt_type_reit = debt_type_reit[1]
coefficient_debt_type_reit = sum_debt_type_reit/sum_type_reit
#print('Процент невозрата клиентов, берущих кредит на недвижимость составляет {:.2%}'.format(coefficient_debt_type_reit))

type_auto = data[data['type_purpose'] == 'автомобиль'].count()
sum_type_auto = type_auto[1]
debt_type_auto = data[(data['type_purpose'] == 'автомобиль')&(data['debt'] == 1)].count()
sum_debt_type_auto = debt_type_auto[1]
coefficient_debt_type_auto = sum_debt_type_auto/sum_type_auto
#print('Процент невозрата клиентов, берущих кредит на авто составляет {:.2%}'.format(coefficient_debt_type_auto))

type_education = data[data['type_purpose'] == 'образование'].count()
sum_type_education = type_education[1]
debt_type_education = data[(data['type_purpose'] == 'образование')&(data['debt'] == 1)].count()
sum_debt_type_education = debt_type_education[1]
coefficient_debt_type_education = sum_debt_type_education/sum_type_education
#print('Процент невозрата клиентов, берущих кредит на образование составляет {:.2%}'.format(coefficient_debt_type_education))

type_wedding = data[data['type_purpose'] == 'свадьба'].count()
sum_type_wedding = type_wedding[1]
debt_type_wedding = data[(data['type_purpose'] == 'свадьба')&(data['debt'] == 1)].count()
sum_debt_type_wedding = debt_type_wedding[1]
coefficient_debt_type_wedding = sum_debt_type_wedding/sum_type_wedding
#print('Процент невозрата клиентов, берущих кредит на свадьбу составляет {:.2%}'.format(coefficient_debt_type_wedding))

data_sellect_type_purpose = [
    ['Должники, берущие на','недвижимость {:.2%}'.format(coefficient_debt_type_reit), 'Лучше на {:.2%}'.format((coefficient_debt_type_reit-coefficient_debt)*-1)],
    ['Должники, берущие на','авто {:.2%}'.format(coefficient_debt_type_auto), 'Хуже на {:.2%}'.format((coefficient_debt_type_auto-coefficient_debt)*1)],
    ['Должники, берущие на','на образование {:.2%}'.format(coefficient_debt_type_education), 'Хуже на {:.2%}'.format((coefficient_debt_type_education-coefficient_debt)*1)],
    ['Должники, берущие на','на свадьбу {:.2%}'.format(coefficient_debt_type_wedding), 'Лучше на {:.2%}'.format((coefficient_debt_type_wedding-coefficient_debt)*-1)]
]

sellect_type_purpos_columns = ['Категория','Подкатегория','Отклонение от среднего']
df_data_sellect_type_purpose= pd.DataFrame(data = data_sellect_type_purpose, columns = sellect_type_purpos_columns)
#print(df_data_sellect_type_purpose)

final_columns = ['Категория','Подкатегория','Отклонение от среднего']
data_final = data_select_children + data_sellect_id_family + data_sellect_income + data_sellect_type_purpose
df_final = pd.DataFrame(data = data_final, columns = final_columns)
#print(df_final)

data_pivot = data.pivot_table(index=['family_status','type_purpose','type_have_children'], columns='type_total_income', values='debt', aggfunc='sum')
print(data_pivot)

type_total_income                                      Высокий  Низкий  \
family_status         type_purpose type_have_children                    
в разводе             автомобиль   0                         3       8   
                                   1                         3       2   
                      недвижимость 0                         8      12   
                                   1                         1      12   
                      образование  0                         1       4   
                                   1                         0       3   
вдовец / вдова        автомобиль   0                         5      10   
                                   1                         1       0   
                      недвижимость 0                         5      10   
                                   1                         2       0   
                      образование  0                         2       7   
                                   1  

### Вывод

Процент невозрата по кредитам на недвижимость составляет 7.23% (лучше среднего на 0.88%)

Процент невозрата по кредитам на авто составляет 9.36% (хуже среднего на 1.24%)

Процент невозрата по кредитам на образование составляет 9.22% (хуже среднего на 1.10%)

Процент невозрата по кредитам на свадьбу составляет 8% (лучше среднего на 0.11%)

### Шаг 4. Общий вывод

Самые надежный заёмщики имеют следующие критерии:
- не имеет детей
- вдовец/вдова
- с доходом выше среднего
- берёт кредит с целью покупки недвижимости

Хотя из сводной таблицы видно, что в абсолютном значении больше всего должников (127):
Имеют следующие признаки: "женат/замужем", берут на "недвижимость", не имеют детей, и имеют низкий доход
Из этого можно предположить, что для создания хорошей системы скоринга не хватает дополнительных признаков,
По которым можно было бы определить потенциально ненадёжных должников, например в данном случае:
Нужно было учитывать сколько люди находились в браке и прочие признаки влияющие на % разводов, это важно, так как
Развод - крайне негавно сказывается на желании отдавать кредит за общую недвижимость. 

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.